In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.append("../")

In [11]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [4]:
import textgen

In [5]:
comments_path = textgen.get_resource_path("toxicity-data-merged-comments-05251530.csv")
comments = pd.read_csv(comments_path, index_col=0)

In [6]:
posts_path = textgen.get_resource_path("toxicity-data-merged-posts-05251530.csv")
posts = pd.read_csv(posts_path, index_col=0)

In [7]:
comments.head()

,index,TextId,CommentId,CommentContent,Toxicity,IsInsult,IsThreat,IsObscene,IsIdentityHate
1,1,freya001,freya051002,認賊作父,4,2.0,NaN,NaN,NaN
2,2,freya001,freya051003,2020\下綠營。2020\滅東廠。\綠色恐怖\再現。\n\n消滅\女暴君 詐騙獨裁鬥爭貪污...,5,3.0,3.0,NaN,3.0
3,3,freya002,freya051004,民進黨羸了，台灣輸了。,2,NaN,2.0,NaN,NaN
4,4,freya002,freya051005,在\妳\經營\下\倒退\的\結果\無庸置疑。,3,NaN,2.0,NaN,NaN
5,5,freya002,freya051006,這個\世界第一厲害\，你\是\世界第一\吧？,3,NaN,NaN,NaN,NaN


In [12]:
with open(textgen.get_resource_path("as_chFreq.pickle"), "rb") as fin:
    chfreq = pickle.load(fin)

In [13]:
import gzip

def load_vectors(fname):
    fin = gzip.open(fname, 'rt', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    vectors = []
    stoi = {}
    for line_i, line in tqdm(enumerate(fin)):        
        tokens = line.rstrip().split(' ')
        if line_i < 100:
            continue
        if not all(ch in chfreq for ch in tokens[0]):
            continue
        stoi[tokens[0]] = len(vectors)        
        vectors.append(list(map(float, tokens[1:])))        
        if line_i > 50000: break
    return stoi, vectors

In [14]:
stoi, vectors = load_vectors("h:/cc.zh.300.vec.gz")

In [15]:
stoi["柯文哲"]

17444

In [16]:
itos = {v: k  for k, v in stoi.items()}

In [17]:
with open(textgen.get_resource_path("word_vectors.pkl"), "wb") as fout:
    pickle.dump((itos, stoi, vectors), fout)